# TimeRNNレイヤを実装する
TimeRNNレイヤは、RNNレイヤを時間方向に結合していくレイヤである

In [1]:
import numpy as np
from common.time_layers import RNN
from common.functions import sigmoid

### [演習]
* 以下のTimeRNNレイヤのクラスを完成させましょう

In [2]:
# ヒント

def test(a, b, c):
    print("a=%s"%a, "b=%s"%b, "c=%s"%c)
    return

params = [1,2,3]
test(*params) # *を変数前につけると、各引数に展開される

a=1 b=2 c=3


In [3]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        """
        Wx : 入力xにかかる重み
        Wh : １時刻前のhにかかる重み
        b : バイアス
        stateful : 中間層の出力を次のミニバッチ に渡す場合はTrueにする
        """
        # パラメータのリスト
        self.params = [Wx, Wh, b]
        
        # 勾配のリスト
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        
        self.layers = None
        self.h, self.dh = None, None
        self.stateful = stateful

    def forward(self, xs):
        """
        順伝播計算
        xs : 配列形状は、(バッチサイズ、時間数、前層のノード数)
        """
        Wx, Wh, b = self.params
        N, T, D = xs.shape # バッチサイズ、時間数、前層のノード数
        D, H = Wx.shape # 入力層のノード数、中間層のノード数

        self.layers = []
        
        # hsは、中間層の出力hを時間方向につなげたもの
        hs = np.empty((N, T, H))

        # 中間層の出力hを初期化する
        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H))

        # 時間方向に計算を進める
        for t in range(T):
            
            # RNNレイヤを定義する
            layer = RNN(*self.params) # *を変数前につけると、各引数に展開される
            
            # 時刻tのデータをRNNレイヤに入力する
            self.h = layer.forward(xs[:, t, :], self.h)
            
            # 中間層の出力hをhsに代入する
            hs[:, t, :] = self.h
            
            # レイヤを追加する
            self.layers.append(layer)

        return hs

    def backward(self, dhs):
        """
        逆伝播計算
        dhs : 各時刻における出力層からの勾配を格納した変数. 配列形状は(バッチ数、時間数、中間層のノード数)
        """
        
        Wx, Wh, b = self.params
        N, T, H = dhs.shape # バッチサイズ、時間数、中間層のノード数
        D, H = Wx.shape # 前層のノード数、　中間層のノード数

        # dxsを初期化する. dxsは、各時刻におけるdxを格納する変数
        dxs = np.empty((N, T, D)) # バッチ数、時間数、前層のノード数
        
        # dhの初期値
        dh = 0
        
        # 勾配の初期値
        grads = [0, 0, 0] #Wxの勾配、 Whの勾配、 bの勾配
        
        # 時間方向と逆向きに計算を進める
        for t in reversed(range(T)):
            
            # RNNレイヤの呼び出し
            layer = self.layers[t]
            
            # RNNレイヤの逆伝播計算
            # RNNレイヤに入力される勾配は、2方向から来るので、2つの値を足す
            dx, dh = layer.backward(dhs[:, t, :] + dh) 

            # dxをdxsに格納する
            dxs[:, t, :] = dx

            # Wxの勾配、 Whの勾配、 bの勾配、をそれぞれ足し合わせる
            for i, grad in enumerate(layer.grads):
                grads[i] += grad
        
        # Wxの勾配、 Whの勾配、 bの勾配、を保持しておく
        for i, grad in enumerate(grads):
            self.grads[i][:] = grad # 同じメモリ位置に代入
            
        # 最後の中間層のdhを保持しておく
        self.dh = dh

        return dxs

In [4]:
D = 1 # 前層のノード数
H = 5 # 中間層のノード数
Wx = np.random.randn(D, H)
Wh = np.random.randn(H, H)
b = np.zeros(H)

# オブジェクトの生成
time_rnn = TimeRNN(Wx, Wh, b)
print("id of time_rnn.grads[0]", id(time_rnn.grads[0]))
print()

# 順伝播計算
N = 4 # バッチサイズ
T = 5 # 時間数
xs = np.random.randn(N, T, D)
hs = time_rnn.forward(xs)
# print("hs=", hs)
# print()

# 逆伝播計算
dhs = np.random.randn(N, T, H)
dxs = time_rnn.backward(dhs)
# print("dxs=", dxs)
# print()

print("id of time_rnn.grads[0]", id(time_rnn.grads[0]))
print()


id of time_rnn.grads[0] 4518698800

id of time_rnn.grads[0] 4518698800

